In [1]:
from pywebio import start_server, config
from pywebio.pin import *
from pywebio.input import *
from pywebio.output import *
import pandas as pd
import openpyxl
import csv
import time
import re
global ingredientsdf, recipedf, stepsdf, ingFilepath, recipeFilepath, stepsFilepath, dfDisplay, filepath,ingredDisplaydf
filepath = "C:\\Users\\kelli\\Downloads\\cookbook.xlsx"
pd.options.display.float_format = '{:,.0f}'.format

In [2]:
def save_dfs():
    recipedf.to_excel(filepath,sheet_name='Recipe')
    ingredientsdf.to_excel(filepath,sheet_name='Ingredients')
    stepsdf.to_excel(filepath,sheet_name='Steps')
    ingredientsdf = pd.read_excel(ingFilepath, sheet_name='Ingredients')
    recipedf = pd.read_excel(recipeFilepath, sheet_name='Recipe')
    stepsdf = pd.read_excel("C:\\Users\\kelli\\Downloads\\cookbook.xlsx", sheet_name='Steps')
    

In [3]:
def view_recipes():
    put_markdown("## Recipes").style('text-align:center')
    put_html(dfDisplay.to_html(border=0))
    optionsList = []
    for i in range(1, len(dfDisplay) + 1):
        optionsList.append(i)
    response = select('Which recipe would you like to view?', options = optionsList)
    display_recipe(response)

In [4]:
def generate_instructions(response):
    stepsDisplaydf = stepsdf.loc[stepsdf['StepListID'] == response]
    stepsDisplaydf = stepsDisplaydf.drop(columns=['StepListID'])
    steps_str = stepsDisplaydf.to_string(header=False,
                            index=False,
                            index_names=False).split('\n')
    for i in range(len(steps_str)):
        steps_str[i] = ' '.join(steps_str[i].split())
    steps_str = '\n'.join(steps_str)
    return steps_str
    

In [5]:
def display_recipe(response):
    ingredDisplaydf = ingredientsdf.loc[ingredientsdf['IngredListID'] == response]
    ingredDisplaydf = ingredDisplaydf.drop(columns=['IngredListID'])
    ingredDisplaydf.index = range(1, len(ingredDisplaydf)+1)
    clear()
    text_recipe = "# {}".format(recipedf.iloc[response - 1]['RecipeName'])
    put_markdown(text_recipe).style('text-align:center')
    #put_html(ingredDisplaydf.to_html(border=0))
    instructionsTxt = generate_instructions(response)
    put_markdown("### Ingredients")
    put_column([put_html(ingredDisplaydf.to_html(border=0)), None, put_text(instructionsTxt)])
    #put_text(instructionsTxt)

In [6]:
def get_ingredients(num):
    inputStr = "Enter Ingredient Information for Ingredient {}".format(num + 1)
    ingred_info = input_group(inputStr, [
        input('Ingredient Name', name = 'Ingredient', type = TEXT),
        input('Ingredient Amount', name = 'Amount', type = TEXT),
        input('Ingredient Measurement(example: cup, teaspoon', name ='Measurement', type = TEXT)
    ])
    ingredHelp = {'IngredListID':len(recipedf)}
    ingred_info.update(ingredHelp)
    return ingred_info

In [7]:
def get_steps(num):
    inputStr = "Enter Step {}".format(num + 1)
    info = input_group(inputStr, [
        input('Instruction', name = 'Instruction', type = TEXT)
    ])
    step = num + 1
    step = str(step) + '.'
    stepsHelp = {'StepListID':len(recipedf),'StepNum': step}
    info.update(stepsHelp)
    return info

In [8]:
def add_recipe():
    clear()
    put_markdown('# Cookbook ').style('text-align:center')
    #get information from user
    info = input_group("Please Enter your Recipe Information",[
        input('Enter Recipe Name', name = 'RecipeName', type = TEXT),
        input('Enter Recipe Description', name = 'Description', type = TEXT),
        radio('Select your Tags', options = ['Vegetarian', 'Heart Healthy', 'Protien Heavy', 'Party'], name = 'tags'),      
        input('Food Category (Example: Indian, American, Italian)', name = 'foodCat', type = TEXT),
        select('Cuisine', options = ['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer'], name  = 'cuisine'),
        input('Enter the Prep Time for the Meal', name = 'prepTime', type = NUMBER),
        input('Enter the Cook Time for the Meal', name = 'cookTime', type = NUMBER),
        input('Enter the number of ingredients in recipe:', name = 'numIngred', type = NUMBER),
        input('Enter the number of steps in recipe:', name = 'numInstruct', type = NUMBER)
    ])
    recipeHelp = {'RecipeID': len(recipedf) + 1}
    info.update(recipeHelp)
    recipedf.loc[len(recipedf)] = info
    for i in range(info['numIngred']):
        ingred = get_ingredients(i)
        ingredientsdf.loc[len(ingredientsdf)] = ingred
    for i in range(info['numInstruct']):
        steps = get_steps(i)
        stepsdf.loc[len(stepsdf)] = steps
    recipedf.index = recipedf.index + 1
    ingredientsdf.index = ingredientsdf.index + 1
    stepsdf.index = stepsdf.index + 1
    global dfDisplay 
    dfDisplay = recipedf.drop(columns=['RecipeID', 'numIngred','numInstruct'])
    dfDisplay.rename(columns={'RecipeName': 'Recipe', 'Description': 'Description', 'tags':'Tag', 'foodCat':'Category', 'cuisine': 'Cuisine', 'prepTime': 'Prep Time', 'cookTime': 'Cook Time'}, inplace=True)
    view_recipes()


In [9]:
def login():
    credentials = input_group("loginInfo", [
        input("Username", name="username"),
        input("Password", name="password",
            type=PASSWORD,
            placeholder="Enter your password",
            help_text="Please contact IT Support for access.",
            required = True
            ),
        ])
    with put_loading(shape='border', color='dark').style('text-align:center'):
        for row in cred_list:
            if (row[0] == credentials['username']) & (row[1] == credentials['password']):

                add_recipe()
            else:
                put_text('Incorrect Username or Password. Please try again')
                time.sleep(5)
                clear()
                login()     

In [10]:
filepath= "C:\\Users\\kelli\\Downloads\\cookbook.xlsx"
ingredientsdf = pd.read_excel(filepath, sheet_name='Ingredients')
recipedf = pd.read_excel(filepath, sheet_name='Recipe')
stepsdf = pd.read_excel(filepath, sheet_name='Steps')
with open("C:\\Users\\kelli\\Downloads\\userPassDB.csv", 'r',encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    cred_list = []
    for row in csv_reader:
        cred_list.append(row)
start_server(add_recipe, port = 8080, debug = True )

Running on all addresses.
Use http://192.168.1.216:8080/ to access the application


RuntimeError: This event loop is already running

In [ ]:
display(recipedf)

In [ ]:
# Need to add label for recipe view
display(stepsdf)

In [ ]:
display(ingredDisplaydf)